In [ ]:
!pip install torch transformers accelerate bitsandbytes datasets torch-tb-profiler peft evaluate

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from evaluate import load
from torch.profiler import profile, record_function, ProfilerActivity
import time

# Load TinyBERT model and tokenizer for GLUE (MNLI task)
model_name = "huawei-noah/TinyBERT_General_4L_312D"  # Change to "huawei-noah/TinyBERT_General_4L_312D" for TinyBERT
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,  # MNLI has 3 classes: "entailment", "neutral", "contradiction"
    ignore_mismatched_sizes=True
)
model.eval()

# Load the GLUE MNLI dataset
dataset = load_dataset("glue", "mnli", split="validation_matched[:100]")  # Use a subset for profiling
metric = load("glue", "mnli")

# Preprocess the input data for MNLI
def preprocess(example):
    inputs = tokenizer(
        example["premise"],
        example["hypothesis"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    inputs["labels"] = torch.tensor(example["label"]).unsqueeze(0)  # Add label tensor
    return inputs

# Process the dataset
inputs = [preprocess(dataset[i]) for i in range(len(dataset))]

# Move model and inputs to GPU (if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Step 1: Baseline Inference Latency Profiling
print("Running inference latency profiling...")
with torch.no_grad():
    start_time = time.time()
    for input_data in inputs:
        input_data = {k: v.to(device) for k, v in input_data.items()}
        outputs = model(**input_data)
    end_time = time.time()
    print(f"Inference Latency: {end_time - start_time:.4f} seconds")

# Step 2: Detailed Profiling with PyTorch Profiler
print("\nRunning detailed profiling...")
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
             record_shapes=True, with_stack=True) as prof:
    with torch.no_grad():
        for input_data in inputs:
            with record_function("model_inference"):
                outputs = model(**{k: v.to(device) for k, v in input_data.items()})

# Print the profiling results
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))
prof.export_chrome_trace("glue_mnli_profiler_trace.json")
print("Profiler data saved to glue_mnli_profiler_trace.json")

# Step 3: Compute Accuracy
print("\nComputing accuracy...")
predictions = []
labels = []

with torch.no_grad():
    for input_data in inputs:
        input_data = {k: v.to(device) for k, v in input_data.items()}
        outputs = model(**input_data)
        logits = outputs.logits
        predictions.append(torch.argmax(logits, dim=-1).cpu().item())
        labels.append(input_data["labels"].cpu().item())

accuracy = metric.compute(predictions=predictions, references=labels)["accuracy"]
print(f"Accuracy on GLUE (MNLI) dataset: {accuracy:.4f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference latency profiling...
Inference Latency: 0.6742 seconds

Running detailed profiling...
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us     628.758ms        73.54%     628.758ms       6.288ms           100  
                                        model_inference        39.82%     260.339ms       100.00%     6

In [ ]:
!pip install wandb -Uq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import wandb
wandb.login()

sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'loss',
    'goal': 'minimize'
    }

sweep_config['metric'] = metric

parameters_dict = {
    # Optimizer Selection
    'optimizer': {
        'values': ['adamw_torch', 'adafactor', 'adamw_hf', 'adamw_8bit']
    },

    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 1e-3
    },

    # Learning Rate Scheduler
    'lr_scheduler': {
        'values': [
            'linear',
            'cosine',
            'cosine_with_restarts',
            'polynomial',
            'constant',
            'constant_with_warmup'
        ]
    },

    # Weight Decay
    'weight_decay': {
        'values': [0.0, 0.01, 0.001, 0.1]
    },

    # Warm-up Steps (as a percentage of total training steps)
    'warmup_ratio': {
        'values': [0.0, 0.05, 0.1, 0.15]
    },

    # Batch Size
    'train_batch_size': {
        'values': [4, 8, 16, 32]
    },

    'gradient_accumulation_steps': {
        'values' : [2, 4, 8, 16, 32]
    },

    # LoRA Hyperparameters
    'lora_r': {
        'values': [8, 16, 32]
    },
    'lora_alpha': {
        'values': [8, 16, 32]
    },
    'lora_dropout': {
        'values': [0.05, 0.1, 0.15]
    }
}

sweep_config['parameters'] = parameters_dict

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="TinyBert with GLUE on Kaggle")

Create sweep with ID: giymvsf9
Sweep URL: https://wandb.ai/garima440-new-york-university/TinyBert%20with%20GLUE%20on%20Kaggle/sweeps/giymvsf9


In [ ]:
import torch
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments
)
from datasets import load_dataset
from evaluate import load
from peft import LoraConfig, get_peft_model
import os
import wandb
from transformers.integrations import WandbCallback
from transformers.trainer_callback import TrainerCallback

# Load TinyBERT model and tokenizer
model_name = "huawei-noah/TinyBERT_General_4L_312D"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,  # MNLI has 3 classes
    ignore_mismatched_sizes=True,
    torch_dtype=torch.float32  # Explicitly set to float32
)

global_best_accuracy = 0
global_best_model_checkpoint = "./global_best_model"

class SaveBestAcrossSweepsCallback(TrainerCallback):
    def __init__(self, metric_name="eval_accuracy"):
        self.metric_name = metric_name
        self.last_eval_metrics = None

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        # Store the latest evaluation metrics
        self.last_eval_metrics = metrics
        print(f"Evaluation Metrics: {metrics}")

    def on_train_end(self, args, state, control, **kwargs):
        global global_best_accuracy, global_best_model_checkpoint

        # Use the stored evaluation metrics
        if self.last_eval_metrics:
            current_accuracy = self.last_eval_metrics.get(self.metric_name, 0)
            print(f"Sweep Evaluation Accuracy: {current_accuracy:.4f}")

            if current_accuracy > global_best_accuracy:
                global_best_accuracy = current_accuracy
                print(f"New global best {self.metric_name} found: {current_accuracy:.4f}")

                # Save the best model and tokenizer
                model = kwargs.get('model')
                if model:
                    try:
                        model.save_pretrained(global_best_model_checkpoint)
                        tokenizer.save_pretrained(global_best_model_checkpoint)

                        # Log the best model to wandb
                        if wandb.run:
                            wandb.save(os.path.join(global_best_model_checkpoint, "*"))

                        print(f"Global best model saved to {global_best_model_checkpoint}")
                    except Exception as e:
                        print(f"Error saving global best model: {e}")
        else:
            print("No evaluation metrics found for this sweep.")

def train(config=None):
    global model
    with wandb.init(config=config):
        config = wandb.config

        lora_config = LoraConfig(
            r=config.lora_r,
            lora_alpha=config.lora_alpha,
            target_modules=["query", "value"],
            lora_dropout=config.lora_dropout,
            bias="none",
            task_type="SEQ_CLS"
        )

        model = get_peft_model(model, lora_config)
        model.to("cuda")


        train_dataset = load_dataset("glue", "mnli", split="train[:20000]")
        validation_dataset = load_dataset("glue", "mnli", split="validation_matched[:5000]")

        def preprocess_function(examples):
            inputs = tokenizer(
                examples["premise"],
                examples["hypothesis"],
                padding="max_length",
                truncation=True,
                max_length=512
            )
            inputs["labels"] = examples["label"]
            return inputs

        train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
        validation_dataset = validation_dataset.map(preprocess_function, batched=True, remove_columns=validation_dataset.column_names)

        training_args = TrainingArguments(
            output_dir="./results",
            num_train_epochs=5,  # Each sweep is 5 epochs
            per_device_train_batch_size=config.train_batch_size,
            per_device_eval_batch_size=32,
            warmup_ratio=config.warmup_ratio,
            weight_decay=config.weight_decay,
            logging_dir="./logs",
            logging_steps=10,
            evaluation_strategy="epoch",
            save_strategy="no",  # Disable default saving
            load_best_model_at_end=False,
            metric_for_best_model="accuracy",
            gradient_accumulation_steps=config.gradient_accumulation_steps,
            learning_rate=config.learning_rate,
            lr_scheduler_type=config.lr_scheduler,
            fp16=True,
            bf16=False
        )

        metric = load("glue", "mnli")

        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = torch.argmax(torch.from_numpy(logits), dim=-1)
            return metric.compute(predictions=predictions, references=labels)

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=validation_dataset,
            compute_metrics=compute_metrics,
            callbacks=[SaveBestAcrossSweepsCallback(), WandbCallback()]
        )

        model.print_trainable_parameters()

        trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
wandb.agent(sweep_id, train, count=20)

wandb: Agent Starting Run: at3hhe3k with config:
wandb: 	gradient_accumulation_steps: 16
wandb: 	learning_rate: 0.0004486513370893436
wandb: 	lora_alpha: 16
wandb: 	lora_dropout: 0.05
wandb: 	lora_r: 16
wandb: 	lr_scheduler: constant
wandb: 	optimizer: adamw_hf
wandb: 	train_batch_size: 8
wandb: 	warmup_ratio: 0.05
wandb: 	weight_decay: 0.001


You are adding a <class 'transformers.integrations.integration_utils.WandbCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback
WandbCallback
SaveBestAcrossSweepsCallback


trainable params: 80,811 || all params: 14,431,998 || trainable%: 0.5599


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: WARNING Config item 'gradient_accumulation_steps' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_ratio' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'gradient_accumulation_steps' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_ratio' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Accuracy
0,0.805800,0.774015,0.738600
1,0.675800,0.658774,0.751800
2,0.618000,0.624771,0.764200
4,0.601600,0.607433,0.764800


...

trainable params: 80,811 || all params: 14,431,998 || trainable%: 0.5599


wandb: WARNING Config item 'gradient_accumulation_steps' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_ratio' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'gradient_accumulation_steps' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_ratio' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Accuracy
0,1.098000,1.098127,0.366800
1,1.096900,1.097542,0.349200
2,1.094700,1.096940,0.353200
3,1.093400,1.096509,0.360400
4,1.094700,1.096423,0.363600


Evaluation Metrics: {'eval_loss': 1.098126769065857, 'eval_accuracy': 0.3668, 'eval_runtime': 10.9573, 'eval_samples_per_second': 456.318, 'eval_steps_per_second': 14.328, 'epoch': 0.9984}
Evaluation Metrics: {'eval_loss': 1.0975420475006104, 'eval_accuracy': 0.3492, 'eval_runtime': 10.9793, 'eval_samples_per_second': 455.404, 'eval_steps_per_second': 14.3, 'epoch': 1.9968}
Evaluation Metrics: {'eval_loss': 1.096940040588379, 'eval_accuracy': 0.3532, 'eval_runtime': 10.9083, 'eval_samples_per_second': 458.366, 'eval_steps_per_second': 14.393, 'epoch': 2.9952}
Evaluation Metrics: {'eval_loss': 1.096509337425232, 'eval_accuracy': 0.3604, 'eval_runtime': 10.8964, 'eval_samples_per_second': 458.865, 'eval_steps_per_second': 14.408, 'epoch': 3.9936}
Evaluation Metrics: {'eval_loss': 1.0964232683181763, 'eval_accuracy': 0.3636, 'eval_runtime': 10.9564, 'eval_samples_per_second': 456.353, 'eval_steps_per_second': 14.329, 'epoch': 4.992}
Sweep Evaluation Accuracy: 0.3636


eval/accuracy,██▁▁▃▃▅▅▇▇
eval/loss,██▆▆▃▃▁▁▁▁
eval/runtime,▆▆██▂▂▁▁▆▆
eval/samples_per_second,▃▃▁▁▇▇██▃▃
eval/steps_per_second,▃▃▁▁▇▇██▃▃
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇▇▇██████
train/grad_norm,▄▄▂▅▅▄▄▅▆▂▂▂▄▄█▃▁▁▄▁▂▂▂▅▅▆▆▂▂▃▂▄▄▄▆▃▄▃▂▂
train/learning_rate,▂▃▅▆▆████▇▇▇▇▇▇▆▆▆▆▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁
train/loss,█████▇▇▇▇▇▆▆▆▇▇▆▅▅▅▅▄▆▅▃▄▂▂▂▂▃▂▂▆▆▅▃▃▁▃▂
eval/accuracy,0.3636


In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments
)
import torch

In [ ]:
global_best_model_checkpoint = "./global_best_model"

In [ ]:
# Save the model
model.save_pretrained(global_best_model_checkpoint)

# Save the tokenizer
tokenizer.save_pretrained(global_best_model_checkpoint)

('./global_best_model/tokenizer_config.json',
 './global_best_model/special_tokens_map.json',
 './global_best_model/vocab.txt',
 './global_best_model/added_tokens.json',
 './global_best_model/tokenizer.json')